# Project Documentation Generator: SQLPage Workout Logger

## Introduction

This Jupyter Notebook is an automated tool designed to generate a comprehensive suite of documentation for the **SQLPage Workout Logger** project. It inspects the project's database, file structure, and source code comments to produce a set of up-to-date markdown reports, fully formatted with YAML frontmatter for use in Obsidian.

The notebook generates the following reports:
1.  **Database Schema Report:** A detailed look at each table, its columns, and sample data.
2.  **Folder Tree Report:** An ASCII representation of the project's file structure.
3.  **SQL Comment Documentation:** A reference guide generated from Javadoc-style comments in your `.sql` files.
4.  **Project Style Guide:** A static document outlining coding and documentation standards.

---

## Table of Contents
1.  [**Configuration**](#1-Configuration)
2.  [**How to Use**](#2-How-to-Use)
3.  [**Report Generation**](#3-Report-Generation)
    - [Part A: Database Schema Report](#Part-A-Database-Schema-Report)
    - [Part B: Folder Tree Report](#Part-B-Folder-Tree-Report)
    - [Part C: SQL Documentation Report](#Part-C-SQL-Documentation-Report)
    - [Part D: Project Style Guide](#Part-D-Project-Style-Guide)

---

## 1. Configuration

All user-configurable settings are located in the first code cell. You must review and update these paths before running the notebook.

- **`project_name`**: The base name used in report titles and filenames (e.g., `"SQLPage - Workout Logger"`).
- **`db_path`**: The full or relative path to the SQLite database file (e.g., `'workouts.db'`).
- **`source_folder_for_tree`**: The path to the project's root folder (e.g., `'./sqlpage'`), used for generating the folder tree and parsing SQL comments.
- **`SAMPLE_ROWS`**: The number of sample data rows to include in the schema report.
- **`export_folder`**: The primary destination folder for the generated markdown reports.
- **`obsidian_folder`**: An optional second destination folder, such as an Obsidian vault. Leave empty (`""`) to disable.

---

## 2. How to Use

1.  Update the variables in the **Configuration** code cell immediately following this section.
2.  Run all cells in the notebook sequentially from top to bottom. Using your editor's "Run All" command is recommended.
3.  The final report files will be saved to the location(s) specified in `export_folder` and `obsidian_folder`.

---

## 3. Report Generation

The subsequent cells perform the automated documentation tasks.

In [1]:
# import all the things! (necessary libraries)
import sqlite3
import os
import datetime
from IPython.display import display, Markdown
import re
from collections import defaultdict

In [2]:
## Cell 1: Configuration
# This cell holds all the settings for your reports.

# --- CORE CONFIGURATION ---
# The project name used in titles and filenames
project_name = "SQLPage - Workout"

# Path to the database file to document
db_path = '/Volumes/Public/Container_Settings/sqlpage/www/workouts.db'

# Path to the project folder to map into an ASCII tree
source_folder_for_tree = '/Volumes/Public/Container_Settings/sqlpage' 

# --- REPORT CONFIGURATION ---
# Number of sample data rows to include in the schema report
SAMPLE_ROWS = 3

# --- EXPORT CONFIGURATION ---
# The primary folder where reports will be saved (e.g., for GitHub)
export_folder = '/Volumes/Public/Container_Settings/sqlpage/reports'

# Second location to save reports (e.g., your Obsidian vault)
obsidian_folder = "/Users/david/Documents/remote/04 - Coding Project Docs"

### Part A: Database Schema
*These cells connect to the database and generate the schema report content.*

In [3]:
# Cell 2: Fetch Table Names

table_names = []
error_message = None

if os.path.exists(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
        table_names = [table[0] for table in cursor.fetchall()]
        conn.close()
        print(f"✅ Found {len(table_names)} tables: {', '.join(table_names)}")
    except sqlite3.Error as e:
        error_message = f"❌ Database error: {e}"
        print(error_message)
else:
    print("Skipping database processing because the path was not found.")

✅ Found 9 tables: ExerciseLibrary, WorkoutLog, ProgressionModels, ProgressionModelSteps, WorkoutTemplates, TemplateExerciseList, UserExerciseProgression, users, sessions


In [4]:
## Cell 3: Generate Schema Report Body
# This cell creates the main content for the database schema report.

schema_report_body = ""

if table_names:
    markdown_parts = []
    conn = sqlite3.connect(db_path)
    
    # Generate Table of Contents
    markdown_parts.append("\n## Table of Contents\n\n")
    for table_name in table_names:
        link_anchor = table_name.lower().replace("_", "-")
        markdown_parts.append(f"- [{table_name}](#{link_anchor})\n")
    
    # Generate Detailed Schemas and Data for each table
    for table_name in table_names:
        cursor = conn.cursor()
        markdown_parts.append(f"\n---\n### {table_name}\n\n")
        markdown_parts.append("**Schema**\n\n")
        markdown_parts.append("| Column Name | Data Type |\n| :---------- | :-------- |\n")
        cursor.execute(f"PRAGMA table_info('{table_name}');")
        for column in cursor.fetchall():
            markdown_parts.append(f"| `{column[1]}` | {column[2]} |\n")
        markdown_parts.append("\n")

        markdown_parts.append(f"**Data Samples (First {SAMPLE_ROWS} Rows)**\n\n")
        try:
            cursor.execute(f'SELECT * FROM "{table_name}" LIMIT {SAMPLE_ROWS};')
            col_headers = [desc[0] for desc in cursor.description]
            sample_data = cursor.fetchall()

            if not sample_data:
                markdown_parts.append("_Table is empty._\n\n")
            else:
                markdown_parts.append(f"| {' | '.join(col_headers)} |\n")
                markdown_parts.append(f"|{' :--- |' * len(col_headers)}\n")
                for row in sample_data:
                    processed_row = [str(item) if item is not None else "NULL" for item in row]
                    markdown_parts.append(f"| {' | '.join(processed_row)} |\n")
                markdown_parts.append("\n")
        except sqlite3.Error as e:
            markdown_parts.append(f"_Could not retrieve data: {e}_\n\n")

    conn.close()
    schema_report_body = "".join(markdown_parts)

In [5]:
## Cell 5: Helper Function for Saving
# This cell defines a reusable function to handle saving files to prevent code duplication.

def save_report(folder_path, filename, content):
    """A helper function to safely save content to a file."""
    if not folder_path:
        print("_Skipping save because path is not set._")
        return
    try:
        os.makedirs(folder_path, exist_ok=True)
        full_path = os.path.join(folder_path, filename)
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✅ Report successfully saved to:\n'{full_path}'")
    except Exception as e:
        print(f"❌ Failed to save report to '{folder_path}': {e}")

In [ ]:
## Cell 6: Assemble and Save Schema Report
# This cell assembles the final schema document with Obsidian frontmatter and saves it.

if schema_report_body:
    date_iso = datetime.date.today().isoformat()
    
    # New filename and title convention
    report_type = "Database Schema Report"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"
    
    summary = f"Schema and data samples for the database powering the {project_name} application."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{db_path}"
categories: Homelab
tags:
  - sqlpage
  - workout
  - database
  - schema
  - documentation
cssclasses:
  - academia
  - academia-rounded
---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""
    full_content = frontmatter + callout + schema_report_body

    save_report(export_folder, filename, full_content)
    if obsidian_folder:
        print("\nSaving copy to Obsidian folder...")
        save_report(obsidian_folder, filename, full_content)
else:
    print("Skipping save for schema report because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-06-15 - SQLPage - Workout - Database Schema Report.md'

Saving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/2025-06-15 - SQLPage - Workout - Database Schema Report.md'


### Part B: Folder Tree
*This cell generates the ASCII tree from the source folder.*

In [7]:
#Cell 7: Generate Folder Tree
sqlpage_tree_string = ""

def build_tree_recursive(dir_path, prefix=""):
    """A recursive generator function to build the ASCII tree lines."""
    ignore_list = {'.DS_Store', '__pycache__', '.git', '.vscode'}
    try:
        items = sorted([item for item in os.listdir(dir_path) if item not in ignore_list])
    except OSError:
        return
    pointers = ['├── '] * (len(items) - 1) + ['└── ']
    for pointer, item in zip(pointers, items):
        yield prefix + pointer + item
        path = os.path.join(dir_path, item)
        if os.path.isdir(path):
            extension = '│   ' if pointer == '├── ' else '    '
            yield from build_tree_recursive(path, prefix + extension)

if os.path.isdir(source_folder_for_tree):
    root_name = os.path.basename(os.path.abspath(source_folder_for_tree))
    tree_lines = [f"{root_name}/"]
    tree_lines.extend(list(build_tree_recursive(source_folder_for_tree)))
    sqlpage_tree_string = "\n".join(tree_lines)
    print("✅ Successfully generated folder tree string.")
else:
    print(f"Skipping tree generation because the source folder '{source_folder_for_tree}' was not found.")

✅ Successfully generated folder tree string.


In [ ]:
## Cell 8: Assemble and Save Folder Tree Report
# This final cell assembles the folder tree report with frontmatter and saves it.

if sqlpage_tree_string:
    date_iso = datetime.date.today().isoformat()

    # New filename and title convention
    report_type = "Folder Tree Report"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"
    
    summary = f"An ASCII tree representation of the file and folder structure for the {project_name} project."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{source_folder_for_tree}"
categories: Homelab
tags:
  - sqlpage
  - workout
cssclasses:
  - academia
  - academia-rounded

---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""
    tree_report_body = f"## Directory Tree\n\n```tree\n{sqlpage_tree_string}\n```"
    full_content = frontmatter + callout + tree_report_body

    save_report(export_folder, filename, full_content)
    if obsidian_folder:
        print("\nSaving copy to Obsidian folder...")
        save_report(obsidian_folder, filename, full_content)
else:
    print("Skipping save for folder tree report because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-06-15 - SQLPage - Workout - Folder Tree Report.md'

Saving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/2025-06-15 - SQLPage - Workout - Folder Tree Report.md'


### Part C: SQL Documentation
*These cells parse the Javadoc-style comments from your `.sql` files.*

In [9]:
## Cell 9: Parse Comments from SQL Files

sql_docs_data = []

javadoc_regex = re.compile(r"/\*\*(.*?)\*/", re.DOTALL)
tag_regex = re.compile(r"@(\w+)\s+(.*?)(?=\s*@\w+|\s*$)", re.DOTALL)

if os.path.isdir(source_folder_for_tree):
    for root, _, files in os.walk(source_folder_for_tree):
        for file in sorted(files):
            if file.endswith(".sql"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                    
                    doc_block_match = javadoc_regex.search(content)
                    if doc_block_match:
                        doc_content = doc_block_match.group(1)
                        
                        tags_dict = defaultdict(list)
                        raw_tags = tag_regex.findall(doc_content)

                        for key, val in raw_tags:
                            # Strip whitespace and remove leading asterisks
                            cleaned_val = val.replace('*', '').strip()
                            # If the line starts with a hyphen (for list items), strip it
                            if cleaned_val.startswith('-'):
                                cleaned_val = cleaned_val.lstrip('- ').strip()
                            # Finally, normalize all remaining whitespace
                            cleaned_val = " ".join(cleaned_val.split())
                            
                            tags_dict[key].append(cleaned_val)
                        
                        cleaned_tags = dict(tags_dict)

                        mtime_timestamp = os.path.getmtime(file_path)
                        mtime_date = datetime.datetime.fromtimestamp(mtime_timestamp).strftime('%Y-%m-%d')
                        cleaned_tags['file_last_modified'] = [mtime_date]

                        cleaned_tags['filepath'] = [file_path]
                        sql_docs_data.append(cleaned_tags)
                except Exception as e:
                    print(f"Could not process file {file_path}: {e}")

print(f"✅ Found and parsed documentation comments from {len(sql_docs_data)} SQL files.")

✅ Found and parsed documentation comments from 17 SQL files.


In [10]:
## Cell 10: Generate SQL Documentation Body
# This cell takes the parsed data from the previous cell and formats it into a readable markdown document body.

sql_docs_body = ""
if sql_docs_data:
    # Sort the data by filepath to ensure a consistent order
    sql_docs_data.sort(key=lambda x: x.get('filepath', [''])[0])
    
    parts = ["## SQL File Documentation\n"]
    for doc in sql_docs_data:
        # Helper function to safely get the first item from a list in the doc dict
        def get_tag(tag_name, default=''):
            return doc.get(tag_name, [default])[0]

        # --- Header and Metadata ---
        filename = os.path.basename(get_tag('filepath', 'N/A'))
        parts.append(f"\n---\n### `{filename}`\n")
        parts.append(f"**Path:** `{get_tag('filepath')}`\n")

        doc_updated = get_tag('last-updated', 'N/A')
        file_modified = get_tag('file_last_modified', 'N/A')
        date_status_emoji = "✅" if doc_updated == file_modified else "⚠️"
        parts.append(f"**Last Updated (doc):** `{doc_updated}` | **File Modified:** `{file_modified}` {date_status_emoji}\n\n")

        # --- Core Description ---
        parts.append(f"**Description:** {get_tag('description', '_No description provided._')}\n")

        # --- Function to format list-based tags ---
        def format_list_tag(tag_key, title):
            if tag_key in doc:
                parts.append(f"\n**{title}:**\n")
                for item in doc[tag_key]:
                    parts.append(f"- {item}\n")
        
        # --- Display all relevant tags in a clean order ---
        format_list_tag('requires', 'Requires')
        format_list_tag('param', 'Parameters')
        format_list_tag('returns', 'Returns')
        format_list_tag('see', 'See Also')
        format_list_tag('note', 'Notes')
        format_list_tag('todo', 'TODO')

    sql_docs_body = "".join(parts)
    print("✅ Successfully generated full SQL documentation body.")

✅ Successfully generated full SQL documentation body.


### Part D: Project Style Guide
*This final cell saves the standalone style guide document.*

In [ ]:
## Cell 11: Assemble and Save SQL Documentation Report
# This final cell assembles the full document with frontmatter and saves your new SQL Documentation report.

if sql_docs_body:
    date_iso = datetime.date.today().isoformat()
    
    # New filename and title convention
    report_type = "SQL Comment Documentation"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"
    
    summary = "A guide to docstring conventions and a summary of all documented SQL files in the project."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{source_folder_for_tree}"
categories: Homelab
tags:
  - sqlpage
  - documentation
  - style-guide
cssclasses:
  - academia
  - academia-rounded  
---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""
    
    toc_parts = ["\n## Table of Contents\n\n"]
    for doc in sql_docs_data:
        filename_in_toc = os.path.basename(doc.get('filepath', ['N/A'])[0])
        link_anchor = filename_in_toc.lower().replace('.', '')
        toc_parts.append(f"- [`{filename_in_toc}`](#{link_anchor})\n")
    toc_body = "".join(toc_parts)
    
    full_content = frontmatter + callout + toc_body + sql_docs_body
    
    save_report(export_folder, filename, full_content)
    if obsidian_folder:
        print("\nSaving copy to Obsidian folder...")
        save_report(obsidian_folder, filename, full_content)
else:
    print("Skipping save for SQL documentation because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-06-15 - SQLPage - Workout - SQL Comment Documentation.md'

Saving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/2025-06-15 - SQLPage - Workout - SQL Comment Documentation.md'
